# Mistral Fine-tuning API

Check out the docs: https://docs.mistral.ai/capabilities/finetuning/

In [ ]:
#!pip install mistralai pandas

## Prepare the dataset

In this example, let’s use the ultrachat_200k dataset. We load a chunk of the data into Pandas Dataframes, split the data into training and validation, and save the data into the required jsonl format for fine-tuning.

In [2]:
import pandas as pd
df = pd.read_json('./data/generated_news_correction.jsonl', lines=True)


df_train=df.sample(frac=0.995,random_state=200)
df_eval=df.drop(df_train.index)

df_train.to_json("news_chunk_train.jsonl", orient="records", lines=True)
df_eval.to_json("news_chunk_eval.jsonl", orient="records", lines=True)

In [3]:
!ls -lh

total 296792
-rw-r--r--  1 pierrebittner  staff   3,5K 21 jui 16:49 Step 2 - synthetize news.py
-rw-r--r--  1 pierrebittner  staff   1,8K 21 jui 17:00 Step 3 - critique.py
-rw-r--r--  1 pierrebittner  staff   6,4K 21 jui 17:30 Step 4 - specific-style.py
-rw-r--r--  1 pierrebittner  staff   6,5K 19 jui 16:19 Untitled.ipynb
drwxr-xr-x  5 pierrebittner  staff   160B 21 jui 17:06 __pycache__
-rw-r--r--  1 pierrebittner  staff   1,1K 21 jui 15:53 concatenate.py
drwxr-xr-x  4 pierrebittner  staff   128B 21 jui 18:10 data
-rw-r--r--@ 1 pierrebittner  staff    39K 21 jui 16:19 mistral_finetune_api.ipynb
-rw-r--r--  1 pierrebittner  staff    48K 21 jui 17:52 mistral_finetune_api_news.ipynb
-rw-r--r--  1 pierrebittner  staff    20K 21 jui 18:10 news_chunk_eval.jsonl
-rw-r--r--  1 pierrebittner  staff   3,5M 21 jui 18:10 news_chunk_train.jsonl
-rw-r--r--  1 pierrebittner  staff   447B 21 jui 11:29 prompts.py
-rw-r--r--  1 pierrebittner  staff   3,3K 19 jui 11:53 reformat_data.py
drwxr-xr-x  4 pie

## Reformat dataset
If you upload this ultrachat_chunk_train.jsonl to Mistral API, you might encounter an error message “Invalid file format” due to data formatting issues. To reformat the data into the correct format, you can download the reformat_dataset.py script and use it to validate and reformat both the training and evaluation data:

In [ ]:
# download the validation and reformat script
!wget https://raw.githubusercontent.com/mistralai/mistral-finetune/main/utils/reformat_data.py

In [4]:
# validate and reformat the training data
!python reformat_data.py news_chunk_train.jsonl

In [5]:
# validate the reformat the eval data
!python reformat_data.py news_chunk_eval.jsonl

In [6]:
df_train.iloc[104]['messages']

[{'role': 'user',
  'content': 'As a "News Article Stylist" following the Economist style guide,\nyour task is to refine and rewrite news articles to ensure they meet the high standards of clarity,\nprecision, and sophistication characteristic of the Economist.\nYou are now given a news article.\nRead the news article carefully and point out all stylistic issues of the given news article according\nto the Economist style guide. Do not rewrite the news article. \n\nChina Announces Plans to Build World\'s Largest Solar Farm\n\nEmily Chen\n\n2023-02-18\n\nChina has announced plans to build the world\'s largest solar farm in the Gobi Desert. The farm will cover an area of over 2,000 square kilometers and will have a capacity of 200 gigawatts. This is enough to power over 100 million homes. The project is part of China\'s goal to reach peak carbon emissions before 2030 and to become carbon neutral by 2060. The project is expected to create over 1 million jobs and to help reduce China\'s dep

## Upload dataset

In [7]:
import os
from mistralai.client import MistralClient

api_key = os.environ.get("MISTRAL_API_KEY")
client = MistralClient(api_key=api_key)

with open("news_chunk_train.jsonl", "rb") as f:
    news_chunk_train = client.files.create(file=("news_chunk_train.jsonl", f))
with open("news_chunk_eval.jsonl", "rb") as f:
    news_chunk_eval = client.files.create(file=("news_chunk_eval.jsonl", f))

In [8]:
import json
def pprint(obj):
    print(json.dumps(obj.dict(), indent=4))

In [9]:
pprint(news_chunk_train)

{
    "id": "ef300892-aab1-4fd5-b2b6-787f236ad6f9",
    "object": "file",
    "bytes": 3690402,
    "created_at": 1718986297,
    "filename": "news_chunk_train.jsonl",
    "purpose": "fine-tune"
}


In [10]:
pprint(news_chunk_eval)

{
    "id": "37fc547a-5f4d-4b17-8420-eed7e05d1534",
    "object": "file",
    "bytes": 20791,
    "created_at": 1718986297,
    "filename": "news_chunk_eval.jsonl",
    "purpose": "fine-tune"
}


## Create a fine-tuning job

In [11]:
from mistralai.models.jobs import TrainingParameters

created_jobs = client.jobs.create(
    model="open-mistral-7b",#"mistral-small-latest", #"open-mistral-7b", 
    training_files=[news_chunk_train.id],
    validation_files=[news_chunk_eval.id],
    hyperparameters=TrainingParameters(
        training_steps=10,
        learning_rate=0.0001,
        )
)

In [12]:
pprint(created_jobs)

{
    "id": "8e2d2916-7a00-470d-b990-aa53a2fadaaf",
    "hyperparameters": {
        "training_steps": 10,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": null,
    "model": "open-mistral-7b",
    "status": "QUEUED",
    "job_type": "FT",
    "created_at": 1718986349,
    "modified_at": 1718986349,
    "training_files": [
        "ef300892-aab1-4fd5-b2b6-787f236ad6f9"
    ],
    "validation_files": [
        "37fc547a-5f4d-4b17-8420-eed7e05d1534"
    ],
    "object": "job",
    "integrations": []
}


In [13]:
import time

retrieved_job = client.jobs.retrieve(created_jobs.id)
while retrieved_job.status in ["RUNNING", "QUEUED"]:
    retrieved_job = client.jobs.retrieve(created_jobs.id)
    pprint(retrieved_job)
    print(f"Job is {retrieved_job.status}, waiting 10 seconds")
    time.sleep(10)



{
    "id": "8e2d2916-7a00-470d-b990-aa53a2fadaaf",
    "hyperparameters": {
        "training_steps": 10,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": null,
    "model": "open-mistral-7b",
    "status": "RUNNING",
    "job_type": "FT",
    "created_at": 1718986349,
    "modified_at": 1718986350,
    "training_files": [
        "ef300892-aab1-4fd5-b2b6-787f236ad6f9"
    ],
    "validation_files": [
        "37fc547a-5f4d-4b17-8420-eed7e05d1534"
    ],
    "object": "job",
    "integrations": [],
    "events": [
        {
            "name": "status-updated",
            "data": {
                "status": "RUNNING"
            },
            "created_at": 1718986350
        },
        {
            "name": "status-updated",
            "data": {
                "status": "QUEUED"
            },
            "created_at": 1718986349
        }
    ],
    "checkpoints": [],
    "estimated_start_time": null
}
Job is RUNNING, waiting 10 seconds
{
    "id": "8e2d2916-7a00-47

In [14]:
# List jobs
jobs = client.jobs.list()
pprint(jobs)

{
    "data": [
        {
            "id": "8e2d2916-7a00-470d-b990-aa53a2fadaaf",
            "hyperparameters": {
                "training_steps": 10,
                "learning_rate": 0.0001
            },
            "fine_tuned_model": "ft:open-mistral-7b:c056c2e4:20240621:8e2d2916",
            "model": "open-mistral-7b",
            "status": "SUCCESS",
            "job_type": "FT",
            "created_at": 1718986349,
            "modified_at": 1718986433,
            "training_files": [
                "ef300892-aab1-4fd5-b2b6-787f236ad6f9"
            ],
            "validation_files": [
                "37fc547a-5f4d-4b17-8420-eed7e05d1534"
            ],
            "object": "job",
            "integrations": []
        },
        {
            "id": "d3a4936b-4f52-498a-a5bd-5580b52a19a0",
            "hyperparameters": {
                "training_steps": 10,
                "learning_rate": 0.0001
            },
            "fine_tuned_model": null,
            "model":

In [15]:
# Retrieve a jobs
retrieved_jobs = client.jobs.retrieve(created_jobs.id)
pprint(retrieved_jobs)


{
    "id": "8e2d2916-7a00-470d-b990-aa53a2fadaaf",
    "hyperparameters": {
        "training_steps": 10,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": "ft:open-mistral-7b:c056c2e4:20240621:8e2d2916",
    "model": "open-mistral-7b",
    "status": "SUCCESS",
    "job_type": "FT",
    "created_at": 1718986349,
    "modified_at": 1718986433,
    "training_files": [
        "ef300892-aab1-4fd5-b2b6-787f236ad6f9"
    ],
    "validation_files": [
        "37fc547a-5f4d-4b17-8420-eed7e05d1534"
    ],
    "object": "job",
    "integrations": [],
    "events": [
        {
            "name": "status-updated",
            "data": {
                "status": "SUCCESS"
            },
            "created_at": 1718986433
        },
        {
            "name": "status-updated",
            "data": {
                "status": "RUNNING"
            },
            "created_at": 1718986350
        },
        {
            "name": "status-updated",
            "data": {
              

## Use a fine-tuned model

In [17]:
from mistralai.models.chat_completion import ChatMessage

chat_response = client.chat(
    model=retrieved_jobs.fine_tuned_model,
    messages=[ChatMessage(role="user", content="You are a 'News Writer'. "
    "Your task is to write a news of 500–1000 words on 'Environment and Climate'."
    "Each news should have a title, content, author, location and date.")]
)

In [18]:
pprint(chat_response)

{
    "id": "9c96b420486b460a9e93c93d6d097747",
    "object": "chat.completion",
    "created": 1718986497,
    "model": "ft:open-mistral-7b:c056c2e4:20240621:8e2d2916",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "Title: \"Renewable Energy Revolution: A Step Towards Sustainable Future\"\n\nContent:\n\nIn a groundbreaking development, the global community has taken a significant stride towards a sustainable future as the adoption of renewable energy sources has reached an unprecedented peak. According to the latest report by the International Energy Agency (IEA), the renewable energy sector has experienced a 13% growth rate this year, marking a new era of energy production.\n\nThe rapid transition from fossil fuels to renewable energy sources is a testament to the global commitment towards combating climate change and ensuring a sustainable future for generations to come. The IEA report high

In [20]:
response_ori = client.chat(
        model="open-mistral-7b",
        messages=[ChatMessage(role="user", content="You are a 'News Writer'. "
    "Your task is to write a news of 500–1000 words on 'Environment and Climate'."
    "Each news should have a title, content, author, location and date.")],
    )

In [21]:
pprint(response_ori)

{
    "id": "d0233dc53afc42eea61d84c7ad0fa3ab",
    "object": "chat.completion",
    "created": 1718986682,
    "model": "open-mistral-7b",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "Title: Rapid Rise in Global Temperature: A Wake-Up Call for Climate Action\n\nContent:\n\nThe world is experiencing an alarming increase in global temperatures, with 2020 recording as the hottest year on record, according to the National Aeronautics and Space Administration (NASA) and the National Oceanic and Atmospheric Administration (NOAA). This concerning trend underscores the urgent need for global climate action.\n\nThe average global temperature in 2020 was 1.62 degrees Celsius above the pre-industrial level, surpassing the previous record set in 2016 by 0.04 degrees Celsius. This rise in temperature has been primarily attributed to human activities, particularly the burning of fossil fuels, deforestati

In [24]:
news_1 = response_ori.choices[0].message.content

In [25]:
news_2 = chat_response.choices[0].message.content

In [33]:
response_comparison = client.chat(
        model="mistral-large-latest",
        messages=[ChatMessage(role="user", content=f"""As a 'News Article Stylist' following the Economist style guide,
        your task is to refine and rewrite news articles to ensure they meet the high standards of clarity,
        precision, and sophistication characteristic of the Economist.
        You are now given to news articles.
        Read the news articles carefully and point which one has the most stylistic issues according
        to the Economist style guide. Do not rewrite the news articles.
        <News_1>{news_1}</News_1>
                      
        <News_2> {news_2}</News_2>""")]
    )

In [34]:
response_comparison.choices[0].message.content

'Both news articles are generally well-written and adhere to the principles of clarity, precision, and sophistication as per the Economist style guide. However, News_1 has a few stylistic issues that need to be addressed:\n\n1. The use of exclamatory phrases such as "alarming increase" and "dire consequences" may be seen as sensational and could be replaced with more neutral language while still conveying the urgency of the situation.\n\n2. The phrase "in turn" is repeated twice in the third paragraph, which can be avoided by rephrasing the sentences.\n\n3. The sentence "The World Meteorological Organization (WMO) has warned that the current pace of temperature rise will lead to catastrophic consequences if not addressed urgently" can be made more concise by removing "if not addressed urgently" as the urgency is already implied.\n\n4. The phrase "immediate and drastic action" in the second last paragraph is repetitive as both words convey a sense of urgency. Choosing one would suffice.

In [38]:
response_critique = client.chat(
        model="mistral-large-latest",
        messages=[ChatMessage(role="user", content=f"""As a "News Article Stylist" following the Economist style guide,
        your task is to refine and rewrite news articles to ensure they meet the high standards of clarity,
        precision, and sophistication characteristic of the Economist.
        You are now given a news article.
        Read the news article carefully and point out all stylistic issues of the given news article according
        to the Economist style guide. Do not rewrite the news article. 
        Finaly grade the news from 1 to 10 based on the level of Economist style compliance.
        
        <News>{news_2}</News>
        
        Critique:
        
        Grade:""")]
    )

In [39]:
response_critique.choices[0].message.content

'Critique:\n\n1. The title is a bit sensational and could be more concise to align with the Economist\'s style.\n2. The phrase "groundbreaking development" in the first sentence is redundant and could be eliminated for clarity.\n3. The sentence "marking a new era of energy production" is vague and could benefit from more specific information or data.\n4. The phrase "rapid transition" could be replaced with a more precise term, such as "shift" or "move."\n5. The phrase "a testament to the global commitment" is somewhat clichéd and could be rephrased for originality.\n6. The sentence "The future belongs to those who invest in a sustainable and renewable energy-driven economy" is a bit opinionated and could be rephrased to present a more neutral stance.\n7. The article lacks direct quotes from experts or authorities, which the Economist often includes for credibility.\n8. The article could benefit from more specific data and statistics to support its claims, especially in the paragraph di

In [40]:
response_critique = client.chat(
        model="mistral-large-latest",
        messages=[ChatMessage(role="user", content=f"""As a "News Article Stylist" following the Economist style guide,
        your task is to refine and rewrite news articles to ensure they meet the high standards of clarity,
        precision, and sophistication characteristic of the Economist.
        You are now given a news article.
        Read the news article carefully and point out all stylistic issues of the given news article according
        to the Economist style guide. Do not rewrite the news article. 
        Finaly grade the news from 1 to 10 based on the level of Economist style compliance.
        
        <News>{news_1}</News>
        
        Critique:
        
        Grade:""")]
    )

In [41]:
response_critique.choices[0].message.content

'The article is generally well-written and informative, but there are a few stylistic issues that need to be addressed according to the Economist style guide:\n\n1. The title is somewhat sensationalist. The Economist prefers more straightforward and descriptive titles that give a clear idea of the article\'s content.\n2. The article uses some emotive language, such as "alarming," "concerning," "devastating," and "catastrophic." The Economist prefers a more neutral and objective tone.\n3. The article uses some clichés, such as "wake-up call" and "dire consequences." The Economist encourages more original and precise language.\n4. The article uses some passive voice, such as "has been attributed" and "have been increasing." The Economist prefers active voice whenever possible.\n5. The article uses some redundant phrases, such as "massive destruction and loss of life" and "more severe and frequent disruptions." The Economist encourages concise and precise language.\n6. The article uses so

## Integration with Weights and Biases
We can also offer support for integration with Weights & Biases (W&B) to monitor and track various metrics and statistics associated with our fine-tuning jobs. To enable integration with W&B, you will need to create an account with W&B and add your W&B information in the “integrations” section in the job creation request:



In [ ]:
from mistralai.models.jobs import WandbIntegrationIn

WANDB_API_KEY = "XXX"

created_jobs = client.jobs.create(
    model="open-mistral-7b",
    training_files=[ultrachat_chunk_train.id],
    validation_files=[ultrachat_chunk_eval.id],
    hyperparameters=TrainingParameters(
        training_steps=100,
        learning_rate=0.0001,
    ),
    integrations=[
        WandbIntegrationIn(
            project="test_ft_api",
            run_name="test",
            api_key=WANDB_API_KEY,
        ).dict()
    ],
)